Jugando con CID.

> Por favor, ejecuta una a una cada celda de códgo para que no falle.

Este ejemplo es para aclarar cómo se obtienen los valores que vemos en la CID Inspector para el ejemplo: 

<https://cid.ipfs.tech/#bafkreifzlpgwving46m34a6exyiaj6a6fwruydheyvwvqv36jel73jyjja>


In [1]:
from src.ipfs_cid_playground import *

cid = "bafkreifzlpgwving46m34a6exyiaj6a6fwruydheyvwvqv36jel73jyjja"

print(f"El CID {cid}")

El CID bafkreifzlpgwving46m34a6exyiaj6a6fwruydheyvwvqv36jel73jyjja


El 1º caracter, la "b", es un multibase, quiere decir que el contenido puede estar codificado en diferentes bases.

Los posibles valores en <https://github.com/multiformats/multibase>

En este ejemplo, la "b" indica que el resto de contenido es base32 y en minúsculas (porque se tiene en cuenta)
Comunmente puede ser:
- 'Q': indica base58btc (CIDv0).
- 'b': base32 (CIDv1, minúsculas).
- 'B': base32 (CIDv1, mayúsculas).
- 'f': base16 (CIDv1, hexadecimal).

Así que quitamos el multibase, la "b", y nos quedamos con el resto del valor que es la concatenación de version + multicodec + multihash 

Igualmente tiene el adjetivo "multi" porque implica que puede tener multiples valores, es simplemente una forma de llamarlo.

In [2]:
cid_body = cid[1:]  # quitar 'b'
print(f"El valor es {cid_body}")


El valor es afkreifzlpgwving46m34a6exyiaj6a6fwruydheyvwvqv36jel73jyjja


Sabemos que es base32 así que tenemos que tenerlo en cuenta para codificarlo a hexadecimal.

Pero antes corregimos el valor para que coincida con el padding (relleno) de base32, agregando "=" hasta completar la longuitud multiple de 8.

In [3]:
missing_padding = (8 - len(cid_body) % 8) % 8
cid_body_padded = cid_body.upper() + "=" * missing_padding
print(f"Corregimos el padding para pasar a base32 {cid_body_padded}")

Corregimos el padding para pasar a base32 AFKREIFZLPGWVING46M34A6EXYIAJ6A6FWRUYDHEYVWVQV36JEL73JYJJA======


Pasamos de base32 a hexadecimal

In [4]:
import base64
cid_bytes = base64.b32decode(cid_body_padded)
cid_hex = cid_bytes.hex()
print(f"En hexadecimal el valor es 0x{cid_hex}")

En hexadecimal el valor es 0x01551220b95bcd6aa1a6e799be03c4be1004f81e2da34c0ce4c56d58577e4917fda70948


Tenemos que extaer version + multicodec + multihash 

Extraemos el valor de versión, que es la posición 1 y 2.

Las posibles versiones lo vemos en: <https://github.com/multiformats/cid?tab=readme-ov-file#versions>

PD: evidentemente es la version 1, porque el multibase "b" ya nos lo indicaba.

In [5]:
cod_version = cid_hex[0:2]
print(f"Versión: 0x{cod_version}")

Versión: 0x01


Ahora extraemos el multicodec, que es la posición 3 y 4.

Los posibles valores en <https://github.com/multiformats/multicodec>

El multicodec describe el formato del contenido apuntado por el CID, no el contenido en sí. Le dice al nodo de IPFS cómo decodificar el bloque, que en ejemplos comunes puede ser:
- raw: bytes sin estructura.
- dag-pb: estructura tipo protobuf, usado en IPFS para representar directorios y archivos en sistemas tipo UnixFS
- dag-cbor: objetos serializados tipo JSON binario.
- dag-json: JSON directo.

In [9]:
# Para mostrar el nombre del multicodec, usamos la libreria multiformats
from multiformats import multicodec

cod_multicodec = cid_hex[2:4]
cod_multicodec_hex_to_dec = int(cod_multicodec,16) # convertir de hexadecimal a entero
codec = multicodec.get(code=cod_multicodec_hex_to_dec)
print(f"Código multicodec: 0x{cod_multicodec}")
print(f"Nombre y descripción: {codec.name}, {codec.description}")

Código multicodec: 0x55
Nombre y descripción: raw, raw binary


Ahora extraemos el multihash: https://github.com/multiformats/multihash

Está compuesto por el código, longuitud y digest (es decir, el valor tras aplicar la función hash)

El código de multihash, posición 5 y 6.

Indica el algoritmo usado para genear el hash.

In [10]:
cod_multihash = cid_hex[4:6]
cod_multihash_int = int(cod_multihash, 16) # pasarlo de hexadecimal a entero
mh = multicodec.get(code=cod_multihash_int)
print(f"Código: 0x{cod_multihash}")
print(f"Nombre: {mh.name}")

Código: 0x12
Nombre: sha2-256


La longuitud del multihash, posición 7 y 8

En teoría el código 0x12 (sha2-256) ya implica 32 bytes, pero se incluye la longitud explícitamente por diseño de Multihash.

In [11]:
len_multihash = cid_hex[6:8]
print(f"Longuitud 0x{len_multihash} o {int(len_multihash,16)} en decimal")

Longuitud 0x20 o 32 en decimal


El valor digest, es decir, el propio hash aplicando la función concreta que se especifica en multicodec.

Posición desde 9 al final.

In [12]:
digestMultiHash_hex = cid_hex[8:]
print(f"Digest: {digestMultiHash_hex}")

Digest: b95bcd6aa1a6e799be03c4be1004f81e2da34c0ce4c56d58577e4917fda70948


Ahora el propio multihash, se puede representa a base32 (la propia multibase) para que sea más legible

In [13]:
digest_bytes = bytes.fromhex(digestMultiHash_hex)

# Construir el multihash: <multihash><length><digest>

hex_length = int(len_multihash, 16)
hex_codMultihasg =  int(cod_multihash, 16)

multihash_bytes = bytes([hex_codMultihasg, hex_length]) + digest_bytes

# Codificar en base32 (sin padding, en minúsculas, con prefijo 'b')
multihash_b32 = "b" + base64.b32encode(multihash_bytes).decode("utf-8").lower().rstrip("=")
print(f"El multibase del valor digest es: {multihash_b32}")

El multibase del valor digest es: bciqlsw6nnkq2nz4zxyb4jpqqat4b4lndjqgojrlnlblx4six7wtqssa


Y con esta información del CID, el nodo de IPFS valida que el hash aplicado al dato binario es igual al valor Digest que tendría que ser.